In [1]:
#| echo: false
#| output: false
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import folium
from folium import plugins
from branca.colormap import LinearColormap
from IPython.display import IFrame
from folium import PolyLine
from shapely import wkt

# DART Station TOD Potential

The creation of the final station ranking and map followed most of the same processes as the ridership and population/development rankings and maps. First, the ridership and population/development dataframes were merged to create a final dataframe with all of the metrics included for each station. Next, MinMaxScaler was used on each of the different metrics to normalize the values and then weight each of the stations. For the final weighting of variables, current population of station areas and current ridership were weighted highest as they provide the most accurate current picture of the station areas and station usage. Average daily stops and population change from 2021 to 2023 were weighted the same and thouhgt of as being good indicators for future potentials at stations based on activity in station area and ability for stations to support increased ridership. New homes built between 2021 to 2023, while still an important metric to consider, was weighted the lowest of the metrics because of the current low focus on transit-adjacent development in the region, which meant that many of the station areas had seen no new residential development during the timeframe being analyzed. 

The final rankings map shows that there is TOD potential around the region. While a good amount of the high-ranking stations are located in downtown Dallas, there are also high ranking stations in other cities and in between cities as well. Some of these high ranking stations are closely linked with institutions such as universities or hospitals. 

The dispersion of high-ranking stations also suggests that TOD development could happen in different forms. For example, downtown Dallas is already highly developed, and while more development can occur what might be as if not more beneficial is infrastructure development to allow the stations to become more accessible to more people. This could take the form of complete streets implementation or new public facilities near stations, among others. In areas farther from dense areas development could take the form of more traditional residential and commercial development. For stations that are linked to other institutions, more tailored forms of development might be required that compliment the existing uses in the area.

In [2]:
#| echo: false
#| output: false
train_routes = gpd.read_file('./data/maps/Rail_Routes/Rail_Routes.shp')

lrt_routes = train_routes[~train_routes['route_shor'].isin(['TRE', '425', '620'])]

lrt_routes = lrt_routes.to_crs(epsg=4326)

print(lrt_routes)

    OBJECTID  shape_id  route_id route_shor                     route_long  \
1          2    143519     25810      GREEN   DART LIGHT RAIL - GREEN LINE   
2          3    143777     25810      GREEN   DART LIGHT RAIL - GREEN LINE   
3          4    143599     25812        RED     DART LIGHT RAIL - RED LINE   
4          5    143826     25812        RED     DART LIGHT RAIL - RED LINE   
5          6    143827     25812        RED     DART LIGHT RAIL - RED LINE   
6          7    143503     25809       BLUE    DART LIGHT RAIL - BLUE LINE   
7          8    143505     25809       BLUE    DART LIGHT RAIL - BLUE LINE   
8          9    143510     25809       BLUE    DART LIGHT RAIL - BLUE LINE   
9         10    143512     25809       BLUE    DART LIGHT RAIL - BLUE LINE   
10        11    143530     25810      GREEN   DART LIGHT RAIL - GREEN LINE   
11        12    143537     25810      GREEN   DART LIGHT RAIL - GREEN LINE   
12        13    143544     25810      GREEN   DART LIGHT RAIL - 

In [3]:
#| echo: false
#| output: false
dart_lrt_stations_final = pd.read_csv('./data/maps/dart_lrt_stations_final.csv')
station_newhomes_pop = pd.read_csv('./data/maps/station_newhomes_pop.csv')

print(dart_lrt_stations_final)
print(station_newhomes_pop)

                   stop_name  Avg. Weekday Ridership (FY23)  \
0                ZOO STATION                       0.023364   
1      8TH & CORINTH STATION                       0.162907   
2       WESTMORELAND STATION                       0.232678   
3            MORRELL STATION                       0.044634   
4        ILLINOIS TC/STATION                       0.124879   
..                       ...                            ...   
59       DFW AIRPORT STATION                       0.184177   
60  DOWNTOWN ROWLETT STATION                       0.149372   
61       CAMP WISDOM STATION                       0.046729   
62        UNT DALLAS STATION                       0.108605   
63      HIDDEN RIDGE STATION                       0.002256   

    Ridership_Change_21-23  stop_count  average_daily_stops  \
0                 0.068881        1329             0.177461   
1                 0.216482        1200             0.152850   
2                 0.139606        2073             0.3

In [4]:
#| echo: false
#| output: false
dart_lrt_stations_all_data = pd.merge(
    dart_lrt_stations_final, station_newhomes_pop,
    on='stop_name',
    how='inner'
)[[
    'stop_name', 'Population_2023', 'Population_Change_2021_2023',
    'New_Homes_2021-2023', 'Avg_Weekday_Ridership_23',
    'Ridership_Change_21_23', 'Avg_Daily_Stops', 'station_location'
]]

dart_lrt_stations_all_data

,stop_name,Population_2023,Population_Change_2021_2023,New_Homes_2021-2023,Avg_Weekday_Ridership_23,Ridership_Change_21_23,Avg_Daily_Stops,station_location
0,ZOO STATION,1381,-272,0,280,45,190.0,POINT (-96.8129035265305 32.7407767291492)
1,8TH & CORINTH STATION,1064,-336,13,1146,285,171.0,POINT (-96.7984815227426 32.74808773077971)
2,WESTMORELAND STATION,2318,-46,0,1579,160,296.0,POINT (-96.8717345396692 32.71976372303652)
3,MORRELL STATION,1454,149,0,412,125,53.0,POINT (-96.8026115229864 32.739865729114705)
4,ILLINOIS TC/STATION,1284,-415,0,910,303,239.0,POINT (-96.805169522767 32.72330772574492)
...,...,...,...,...,...,...,...,...
59,DFW AIRPORT STATION,0,0,0,1278,385,55.0,POINT (-97.0395115938477 32.9073917504982)
60,DOWNTOWN ROWLETT STATION,2061,231,29,1062,191,53.0,POINT (-96.5633954722268 32.904116771139606)
61,CAMP WISDOM STATION,2240,-683,0,425,233,211.0,POINT (-96.7881075156358 32.666643715694214)
62,UNT DALLAS STATION,2240,-683,0,809,55,53.0,POINT (-96.8010985184663 32.6540627129614)


In [5]:
#| echo: false
#| output: false
dart_lrt_stations_all_data['Pop_23'] = dart_lrt_stations_all_data['Population_2023'].copy()
dart_lrt_stations_all_data['Pop_Change_21_23'] = dart_lrt_stations_all_data['Population_Change_2021_2023'].copy()
dart_lrt_stations_all_data['New_Homes_21_23'] = dart_lrt_stations_all_data['New_Homes_2021-2023'].copy()
dart_lrt_stations_all_data['Avg_Weekday_Riders_23'] = dart_lrt_stations_all_data['Avg_Weekday_Ridership_23'].copy()
dart_lrt_stations_all_data['Rider_Change_21_23'] = dart_lrt_stations_all_data['Ridership_Change_21_23'].copy()
dart_lrt_stations_all_data['Avg_Stops_Daily'] = dart_lrt_stations_all_data['Avg_Daily_Stops'].copy()

scaler = MinMaxScaler()
dart_lrt_stations_all_data[['Population_2023', 'Population_Change_2021_2023', 'New_Homes_2021-2023', 'Avg_Weekday_Ridership_23', 'Ridership_Change_21_23', 'Avg_Daily_Stops']] = scaler.fit_transform(
    dart_lrt_stations_all_data[['Population_2023', 'Population_Change_2021_2023', 'New_Homes_2021-2023', 'Avg_Weekday_Ridership_23', 'Ridership_Change_21_23', 'Avg_Daily_Stops']]
)

dart_lrt_stations_all_data['Weighted_Score'] = (0.5 * dart_lrt_stations_all_data['Population_2023'] + 
                                          0.35 * dart_lrt_stations_all_data['Population_Change_2021_2023'] + 
                                          0.25 * dart_lrt_stations_all_data['New_Homes_2021-2023'] +
                                          0.5 * dart_lrt_stations_all_data['Avg_Weekday_Ridership_23'] +
                                          0.35 * dart_lrt_stations_all_data['Ridership_Change_21_23'] +
                                          0.35 * dart_lrt_stations_all_data['Avg_Daily_Stops'])

dart_lrt_stations_all_data['Rank'] = dart_lrt_stations_all_data['Weighted_Score'].rank(ascending=False, method='min')

dart_lrt_stations_areas_ranked = dart_lrt_stations_all_data.sort_values(by='Rank')

dart_lrt_stations_areas_ranked[['stop_name', 'Pop_23', 'Pop_Change_21_23', 'New_Homes_21_23', 'Avg_Weekday_Riders_23', 'Rider_Change_21_23', 'Avg_Stops_Daily', 'Weighted_Score', 'Rank', 'station_location']]

,stop_name,Pop_23,Pop_Change_21_23,New_Homes_21_23,Avg_Weekday_Riders_23,Rider_Change_21_23,Avg_Stops_Daily,Weighted_Score,Rank,station_location
9,WEST END STATION,3035,270,136,6341,1205,825.0,1.989808,1.0,POINT (-96.8054505265329 32.78092073745632)
11,ST PAUL STATION,3035,270,136,2790,942,292.0,1.405457,2.0,POINT (-96.7970335244551 32.78430173848197)
12,PEARL/ARTS DISTRICT STATION,2923,294,0,3908,1559,336.0,1.391337,3.0,POINT (-96.79431752441529 32.786623738479214)
10,AKARD STATION,1723,440,31,2780,1100,542.0,1.249016,4.0,POINT (-96.800640525795 32.781898737514496)
8,CONVENTION CENTER STATION,4271,517,28,515,173,221.0,1.033996,5.0,POINT (-96.8029995258667 32.77248673576151)
...,...,...,...,...,...,...,...,...,...,...
0,ZOO STATION,1381,-272,0,280,45,190.0,0.381201,60.0,POINT (-96.8129035265305 32.7407767291492)
62,UNT DALLAS STATION,2240,-683,0,809,55,53.0,0.358962,61.0,POINT (-96.8010985184663 32.6540627129614)
43,MARKET CENTER STATION,0,0,0,481,204,117.0,0.306647,62.0,POINT (-96.8237205324517 32.804640740992795)
51,BELT LINE STATION,0,0,0,273,43,55.0,0.227125,63.0,POINT (-96.9865125789611 32.88802874954939)


In [6]:
#| echo: false
#| output: false
dart_lrt_stations_areas_ranked['station_location'] = dart_lrt_stations_areas_ranked['station_location'].apply(wkt.loads)

dart_lrt_stations_ranked = gpd.GeoDataFrame(
    dart_lrt_stations_areas_ranked[['stop_name', 'Pop_23', 'Pop_Change_21_23', 'New_Homes_21_23', 
                                    'Avg_Weekday_Riders_23', 'Rider_Change_21_23', 'Avg_Stops_Daily', 
                                    'Weighted_Score', 'Rank', 'station_location']], 
    geometry='station_location', crs="EPSG:4326"
)

dart_lrt_stations_ranked

,stop_name,Pop_23,Pop_Change_21_23,New_Homes_21_23,Avg_Weekday_Riders_23,Rider_Change_21_23,Avg_Stops_Daily,Weighted_Score,Rank,station_location
9,WEST END STATION,3035,270,136,6341,1205,825.0,1.989808,1.0,POINT (-96.80545 32.78092)
11,ST PAUL STATION,3035,270,136,2790,942,292.0,1.405457,2.0,POINT (-96.79703 32.78430)
12,PEARL/ARTS DISTRICT STATION,2923,294,0,3908,1559,336.0,1.391337,3.0,POINT (-96.79432 32.78662)
10,AKARD STATION,1723,440,31,2780,1100,542.0,1.249016,4.0,POINT (-96.80064 32.78190)
8,CONVENTION CENTER STATION,4271,517,28,515,173,221.0,1.033996,5.0,POINT (-96.80300 32.77249)
...,...,...,...,...,...,...,...,...,...,...
0,ZOO STATION,1381,-272,0,280,45,190.0,0.381201,60.0,POINT (-96.81290 32.74078)
62,UNT DALLAS STATION,2240,-683,0,809,55,53.0,0.358962,61.0,POINT (-96.80110 32.65406)
43,MARKET CENTER STATION,0,0,0,481,204,117.0,0.306647,62.0,POINT (-96.82372 32.80464)
51,BELT LINE STATION,0,0,0,273,43,55.0,0.227125,63.0,POINT (-96.98651 32.88803)


In [7]:
#| echo: false
max_rank = dart_lrt_stations_ranked['Rank'].max()
min_rank = dart_lrt_stations_ranked['Rank'].min()

colormap = LinearColormap(
    colors=['#5c1c95', '#9342db', '#c69cec'],
    vmin=min_rank, vmax=max_rank
)

def get_color_for_rank(rank):
    return colormap(rank)

m = folium.Map(location=[32.7767, -96.7970], zoom_start=12)

folium.TileLayer('cartodb positron').add_to(m)

for _, row in lrt_routes.iterrows():
    route_geometry = row['geometry']
    
    if route_geometry.geom_type == 'MultiLineString':
        for line in route_geometry.geoms:
            folium.PolyLine(
                locations=[(coord[1], coord[0]) for coord in line.coords],
                color='#A8A8A8',
                weight=2,
                opacity=0.7
            ).add_to(m)
    else:
        folium.PolyLine(
            locations=[(coord[1], coord[0]) for coord in route_geometry.coords],
            color='#A8A8A8',
            weight=2,
            opacity=0.7
        ).add_to(m)
        
for _, row in dart_lrt_stations_ranked.iterrows():
    lat = row['station_location'].y
    lon = row['station_location'].x
    
    popup_content = f"""
        <strong style="font-size: 16px;">{row['stop_name']}</strong><br>
        <span style="font-size: 14px;">Rank: {row['Rank']}</span><br>
        <span style="font-size: 12px;"><em>Population 2023</em>: {row['Pop_23']}</span><br>
        <span style="font-size: 12px;"><em>Population Change 2021-2023</em>: {row['Pop_Change_21_23']}</span><br>
        <span style="font-size: 12px;"><em>New Homes Built 2021-2023</em>: {row['New_Homes_21_23']}</span><br>
        <span style="font-size: 12px;"><em>Average Weekday Ridership 2023 (# of riders)</em>: {row['Avg_Weekday_Riders_23']}</span><br>
        <span style="font-size: 12px;"><em>Ridership Change 2021-2023 (# of riders)</em>: {row['Rider_Change_21_23']}</span><br>
        <span style="font-size: 12px;"><em>Average Daily Service (# of stops)</em>: {row['Avg_Stops_Daily']}</span>
    """
    
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        color=get_color_for_rank(row['Rank']),
        fill=True,
        fill_color=get_color_for_rank(row['Rank']),
        fill_opacity=1,
        popup=folium.Popup(popup_content, max_width=300)
    ).add_to(m)

title_html = """
    <div style="position: absolute; 
                top: 10px; 
                right: 10px; 
                font-size: 14px; 
                font-weight: bold; 
                background-color: white; 
                padding: 5px 10px; 
                border-radius: 5px; 
                border: 1px solid #ccc; 
                z-index: 1000;">
        DART Lightrail Stations Ranked by TOD Development Potential
    </div>
"""

m.get_root().html.add_child(folium.Element(title_html))
m

### Current Shortcomings and Further Analysis

While the current model considers a good amount of metrics in determining TOD potential at DART stations, the analysis is based primarily on current conditions and trends from the immediate past. The model might be strengthened by considering more historical trends, which could paint a longer term picture of the station areas and also help to complete predictive modeling for certain trends such as development and station usage that could better guide where TOD development is prioritized. Predictive modeling was explored with the current data, but it was determined the conclusions would likely not be too reliable due to extent of the data being analyzed. It was challenging to find more historic data for some of the metrics being analyzed, such as new home construction and ridership totals, which is why the datasets were limited to just recent years. 

Other metrics for further analysis could includ economic data and land use data. Economic data could help measure job creation and identify job centers, which is an important consideration when planning transit. I explored using OnTheMap data to perform economic analysis, but economic data is hard to find and sometimes missing for census tracts and census block groups in this region, which made it hard to perform analysis, but is still worth a deeper dive into to help improve the diversity of the model. Similarly, Dallas' current land use maps are outdated and other areas in the region are even more so. Accurate land use analysis would likely require coordinated survey work to create accurate up to date land use maps or using multiple data sources such as OpenStreetMap and sattelite data as a workaround, but would still be useful analysis for determing metrics such as vacant and developable land around stations.

### Conclusion

Supporting transit-oriented development in the Dallas region could help grow the DART system and promote sustainable methods of travel instead of driving. While Dallas is still a car-reliant region, the current transit system does connect a good amount of important locations in the region. The current findings show that there is TOD potential around the region along transit routes and in areas with different built conditions, which means that development could happen in a variety of different ways and create unique and varied, yet connected communities. 